<img src='./img/opengeohub_logo.png' alt='OpenGeoHub Logo' align='right' width='15%'></img>

<a href="./00_index.ipynb"><< Index - Dashboarding with Jupyter Notebooks and Voila </a><span style="float:right;"><a href="./02_voila_dashboards.ipynb"> 02 - Introduction to Voila dashboards >></a></span>

# 3 - Let's get interactive with Jupyter widgets

* [What are Jupyter widgets?](#about)
* [Installing Jupyter widgets](#installation)
* [Jupyter widgets - an overview](#overview)
* [Jupyter widgets in action](#action)

<hr>

## Example 1 - Interactive climate graph application with `widget events`

Required libraries:
* [Ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/)
* [Plotly](https://plot.ly/) for interactive visualization
* [Widgets](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html)

#### Load required libraries

In [83]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, FullScreenControl, Marker
import ipyleaflet

from IPython.display import display, clear_output
import ipywidgets as widgets
import numpy as np
import ee

import plotly.graph_objs as go


### Load ERA5 precipitation and 2m air temperature data from Google Earth Engine data catalogue

#### Initialise EarthEngine

The following command initialises the EarthEngine Python API.

In [2]:
ee.Initialize()

#### Load ERA5 monthly ImageCollection 

With the `ee.ImageCollection` constructor, you can construct a data object with data from the Earth Engine data catalogue. Let us load the ImageCollection of the monthly aggregated ERA5 reanalysis data.

In [3]:
era5_monthly = ee.ImageCollection('ECMWF/ERA5/MONTHLY')

#### Process mean precipitation for each month based on entire time series and build an `ImageCollection` of the resulting image list

From the entire data period (1979 to 2020), let us create the average for each month. For this we have to apply the `ImageCollection.reduce()` function.

If a reducer function is applied to an image collection, the output looses its projections information, as a collection can store images with different projections. For this reason, you have to set the projection again, after the `reduce` function was applied.

You can select and the projection information from the first image of the `ImageCollection` with the `select(i).projection()` function.

In [84]:
era5_monthly_img = era5_monthly.limit(1).first()
collection_img_proj = era5_monthly_img.select(0).projection()
#era5_monthly_img.getInfo()

Now, you can loop over a list of 12 integer values (for each month one) and filter the specific months of each year. On the resulting list, a `reducer` function is applied and the outcome is appended to an empty list.

In [85]:
months = range(1,13)

# Store images in a list
img_list = []
for i in months:
    collection_filtered = era5_monthly.filter(ee.Filter.calendarRange(i,i, 'month'))
    collection_red = collection_filtered.reduce(ee.Reducer.mean())
    
    collection_red_proj = collection_red.setDefaultProjection(collection_img_proj)
    img_list.append(collection_red_proj)
    
#img_list[0].getInfo()

As a final step, you can build an ImageCollection out of the image list with the function `ee.ImageCollection.fromImages()`.

In [6]:
meanMonths_collection = ee.ImageCollection.fromImages(img_list)
#meanMonths_collection.getInfo()

### Visualize an interactive climate graph with `Plotly`

#### Select the temperature and precipitation time-series based on a point feature 

With `ee.Geometry.Point()`, you can define a point feature with EarthEngine. Based on this point location, you can select the two ERA5 variables `total_precipitation_mean` and `mean_2m_air_temperature_mean`.

The result is a list of lists containing the the information `id`, `longitude`, `latitude`, `time` and `temperature`.

In [92]:
point = ee.Geometry.Point(lon.value,lat.value)
tp_point = meanMonths_collection.select('total_precipitation_mean').getRegion(point,500).getInfo()
t2m_point = meanMonths_collection.select('mean_2m_air_temperature_mean').getRegion(point,500).getInfo()
t2m_point

[['id', 'longitude', 'latitude', 'time', 'mean_2m_air_temperature_mean'],
 ['0', 88.99834098593129, -47.00210145334366, None, 281.43853759765625],
 ['1', 88.99834098593129, -47.00210145334366, None, 281.7098693847656],
 ['2', 88.99834098593129, -47.00210145334366, None, 281.8004150390625],
 ['3', 88.99834098593129, -47.00210145334366, None, 281.2383117675781],
 ['4', 88.99834098593129, -47.00210145334366, None, 280.5421142578125],
 ['5', 88.99834098593129, -47.00210145334366, None, 279.7392272949219],
 ['6', 88.99834098593129, -47.00210145334366, None, 279.294189453125],
 ['7', 88.99834098593129, -47.00210145334366, None, 278.8662414550781],
 ['8', 88.99834098593129, -47.00210145334366, None, 278.9442443847656],
 ['9', 88.99834098593129, -47.00210145334366, None, 279.1440734863281],
 ['10', 88.99834098593129, -47.00210145334366, None, 279.9907531738281],
 ['11', 88.99834098593129, -47.00210145334366, None, 280.9185485839844]]

#### Select the data series and convert the variable units

Let us select only the temperature and precipitation data series. During selection, you can directly convert the variable units:
- the precipitation values from `m` to `mm` by multiplying with 1000
- the 2m air temperature values from `K` to `degC` by substracting 273.15

In [103]:
ydata_tp = [row[4]*1000 for row in tp_point[1:]]
    
ydata_t2m = [row[4]-273.15 for row in t2m_point[1:]]

#### Define the plot and plot the climate graph 

For `precipitation`, we define a `barplot` and for `temperature` we define a `lineplot`. With the `data` and `layout` kwargs, we can bring everything together. 

The result is a climate graph for the specified location.

In [105]:
tp = go.Bar(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_tp,
        name='Total precipitation in mm',
        marker=dict(
            color='rgb(204,204,204)',
        ))
    
t2m = go.Scatter(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_t2m,
        name="2m air temperature in deg C",
        yaxis='y2')

data = [tp,t2m]
layout = go.Layout(
        title='Climate graph at location '+ str(round(lat.value,2)) + ' / '+ str(round(lon.value,2)) + ' (lat/lon)',
        yaxis=dict(
            title="Total precipitation in mm"
        ),
        yaxis2=dict(
            title="2 m air temperature in degC",
            overlaying='y',
            side='right',
            range=[0,max(ydata_t2m)+2]
        ),
        plot_bgcolor='white'
    )

fig = go.Figure(data=data, layout=layout)
fig

Let us define a function called `visualize_climate_graph` for the visualization, as we might want to re-use it later.

In [154]:
def visualize_climate_graph():    
    point = ee.Geometry.Point(lon.value, lat.value)
    tp_point = meanMonths_collection.select('total_precipitation_mean').getRegion(point,500).getInfo()
    t2m_point = meanMonths_collection.select('mean_2m_air_temperature_mean').getRegion(point,500).getInfo()

    ydata_tp = [row[4]*1000 for row in tp_point[1:]]
    ydata_t2m = [row[4]-273.2 for row in t2m_point[1:]]

    tp = go.Bar(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_tp,
        name='Total precipitation in mm',
        marker=dict(
            color='rgb(204,204,204)',
        ))
    
    t2m = go.Scatter(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_t2m,
        name="2m air temperature in deg C",
        yaxis='y2')

    data = [tp,t2m]
    layout = go.Layout(
        title='Climate graph at location '+ str(round(lat.value,2)) + ' / '+ str(round(lon.value,2)) + ' (lat/lon)',
        yaxis=dict(
            title="Total precipitation in mm"
        ),
        yaxis2=dict(
            title="2 m air temperature in degC",
            overlaying='y',
            side='right',
            range=[0,max(ydata_t2m)+2]
        ),
        plot_bgcolor='white'
    )

    fig = go.Figure(data=data, layout=layout)
    return fig

### Combine the climate graph with `ipywidgets` to make it dynamic

Let us develop a small application where we have two different `FloatSliders` with the range of latitude and longitude ranges. As soon as the sliders are modified, the climate graph shall be updated as well.

The first step is to define two `FloatSliders`:

In [148]:
lat = widgets.FloatSlider(
    value=53,
    min=-90.0,
    max=90.0,
    step=0.25,
    description='Latitude:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal', #horizontal
    readout=True
)
lat

FloatSlider(value=53.0, continuous_update=False, description='Latitude:', max=90.0, min=-90.0, step=0.25)

In [149]:
lon = widgets.FloatSlider(
    value=20,
    min=-180.0,
    max=180.0,
    step=0.25,
    description='Longitude:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
lon

FloatSlider(value=20.0, continuous_update=False, description='Longitude:', max=180.0, min=-180.0, step=0.25)

Now we define a function called `click()` that calls the visualization function `visualize_climate_graph()`.

In [155]:
def click(b):
    fig = visualize_climate_graph()
    with out:
        clear_output(wait=True)
        fig.show()

In [146]:
lat.observe(click, 'value')
lat

In [156]:
out=widgets.Output()
button=widgets.Button(description='Plot climate graph')
button.on_click(click)

display(lat,lon)


display(button)

display(out)

FloatSlider(value=-52.75, continuous_update=False, description='Latitude:', max=90.0, min=-90.0, step=0.25)

FloatSlider(value=121.0, continuous_update=False, description='Longitude:', max=180.0, min=-180.0, step=0.25)

Button(description='Plot climate graph', style=ButtonStyle())

Output()

In [11]:
from ipywidgets import interact, interactive, interactive_output, fixed, interact_manual


In [34]:
map1 = ipyleaflet.Map(zoom=2)
mark = ipyleaflet.Marker(location=(lat.value, lon.value))
map1.add_layer(mark)
map1

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

<br>

In [75]:
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
#        global lat, lon
#
        mark = ipyleaflet.Marker(location=kwargs.get('coordinates'))
#        layer_group=LayerGroup(layers=(mark,mark))
        map2.add_layer(mark)
#        location = mark.location
#        print(location)
#        lat, lon = location[0], location[1]        

map2.on_interaction(
#    map1.remove_layer(mark),
    handle_click
    )

map2

Map(bottom=712.0, center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

AttributeError: 'Map' object has no attribute 'Marker'

AttributeError: 'Map' object has no attribute 'Marker'

AttributeError: 'Map' object has no attribute 'Marker'

AttributeError: 'Map' object has no attribute 'Marker'

AttributeError: 'Map' object has no attribute 'Marker'

AttributeError: 'Map' object has no attribute 'Marker'

TypeError: changeMarker() missing 1 required positional argument: 'lon'

<a href="./00_index.ipynb"><< Index - Dashboarding with Jupyter Notebooks and Voila </a><span style="float:right;"><a href="./02_voila_dashboards.ipynb"> 02 - Introduction to Voila dashboards >></a></span>

<hr>
&copy; 2020 | Julia Wagemann
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>